# Building a Track Bounds Alert System

This notebook teaches you how to **build** a Track Bounds Alert System from scratch. You'll learn:

1. Understanding the problem and data structures
2. Converting perception to Frenet frame
3. Comparing with ground truth bounds
4. Implementing the alert logic
5. Using the built-in visualization tools

## Problem Statement

We want to detect when perceived track boundaries deviate significantly from the ground truth map. This is critical for:
- **Localization validation**: Is our car where we think it is?
- **Safety systems**: Are we about to leave the track?
- **Perception quality monitoring**: Is our sensor data reliable?

In [ ]:
import sys

sys.path.insert(0, '../../src')

import matplotlib.pyplot as plt
import numpy as np

from simple_autonomous_car import (
    AlertVisualizer,
    Car,
    CarState,
    FrenetMap,
    GroundTruthMap,
    PerceivedMap,
    Sensor,
    Track,
)

## Step 1: Setup - Understanding the Data Structures

Let's create all the components we need and understand what each one does.

In [ ]:
# ============================================================================
# TRACK: The racing track with centerline and boundaries
# ============================================================================
# Parameters:
#   - length: Overall length of the track (meters)
#   - width: Overall width of the track (meters)
#   - track_width: Width of the drivable area (meters)
#   - num_points: Number of points along the centerline (more = smoother)
track = Track.create_simple_track(
    length=80.0,      # Track is 80m long
    width=40.0,       # Track is 40m wide
    track_width=5.0,  # Drivable area is 5m wide
    num_points=200    # 200 points for smooth curves
)

print(f"✓ Track created with {len(track.centerline)} centerline points")
print(f"  Track bounds: {len(track.inner_bound)} inner, {len(track.outer_bound)} outer points")

### Visualization: Let's see the track!

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(track.centerline[:, 0], track.centerline[:, 1], 'b--', linewidth=2, label='Centerline', alpha=0.7)
ax.plot(track.inner_bound[:, 0], track.inner_bound[:, 1], 'k-', linewidth=2.5, label='Inner Bound (Left)')
ax.plot(track.outer_bound[:, 0], track.outer_bound[:, 1], 'k-', linewidth=2.5, label='Outer Bound (Right)')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('Track Layout - Ground Truth Map')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

print("\nTrack visualization shows:")
print("  - Blue dashed line: Centerline (reference path)")
print("  - Black solid lines: Track boundaries (inner and outer)")
print("  - The area between boundaries is the drivable track")

In [ ]:
# ============================================================================
# CAR: The vehicle we're simulating
# ============================================================================
# We place the car at the start of the track
start_point, start_heading = track.get_point_at_distance(0.0)

car = Car(
    initial_state=CarState(
        x=start_point[0],      # X position in world frame
        y=start_point[1],      # Y position in world frame
        heading=start_heading, # Heading angle (radians, 0 = east)
        velocity=8.0            # Initial velocity (m/s)
    ),
    wheelbase=2.5,              # Distance between front and rear axles (meters)
    max_velocity=25.0,          # Maximum velocity (m/s)
    max_steering_angle=np.pi/4  # Maximum steering angle (45 degrees)
)

print(f"✓ Car created at position ({car.state.x:.2f}, {car.state.y:.2f})")
print(f"  Heading: {np.degrees(car.state.heading):.1f} degrees")
print(f"  Velocity: {car.state.velocity:.2f} m/s")

In [ ]:
# ============================================================================
# GROUND TRUTH MAP: The perfect, accurate map of the track
# ============================================================================
# This represents what the track ACTUALLY looks like (perfect knowledge)
# In reality, we'd have this from high-definition mapping
ground_truth_map = GroundTruthMap(track)

print("✓ Ground truth map created")
print("  This is our reference - the 'truth' we compare against")

In [ ]:
# ============================================================================
# PERCEIVED MAP: The noisy, imperfect perception of the track
# ============================================================================
# This simulates what sensors actually see (with errors and noise)
# Parameters:
#   - position_noise_std: How much our position estimate is off (meters)
#   - orientation_noise_std: How much our heading estimate is off (radians)
#   - measurement_noise_std: Noise in sensor measurements (meters)
perceived_map = PerceivedMap(
    ground_truth_map,
    position_noise_std=0.15,      # ±15cm position error
    orientation_noise_std=0.08,   # ±0.08 rad (~4.6°) heading error
    measurement_noise_std=0.3,    # ±30cm measurement noise
)

print("✓ Perceived map created")
print("  This simulates real-world sensor imperfections:")
print(f"    - Position uncertainty: ±{perceived_map.position_noise_std*100:.0f}cm")
print(f"    - Orientation uncertainty: ±{np.degrees(perceived_map.orientation_noise_std):.1f}°")
print(f"    - Measurement noise: ±{perceived_map.measurement_noise_std*100:.0f}cm")

In [ ]:
# ============================================================================
# SENSOR: Simulates a 360° LiDAR or similar sensor
# ============================================================================
# The sensor 'sees' the track boundaries and returns a point cloud
# Parameters:
#   - max_range: Maximum sensor range (meters) - how far it can see
#   - angular_resolution: Angular spacing between sensor rays (radians)
#   - point_noise_std: Additional noise on each point measurement (meters)
sensor = Sensor(
    ground_truth_map,      # Needs ground truth to simulate what it sees
    perceived_map,         # Adds perception errors
    max_range=40.0,        # Can see up to 40m away
    angular_resolution=0.1, # 0.1 rad (~5.7°) between rays
    point_noise_std=0.1,    # ±10cm noise on each point
)

print("✓ Sensor created")
print(f"  Max range: {sensor.max_range}m (360° field of view)")
print(f"  Angular resolution: {np.degrees(sensor.angular_resolution):.1f}°")
print(f"  Point noise: ±{sensor.point_noise_std*100:.0f}cm per measurement")
print("\n  The sensor returns PerceptionPoints - a vector of [x, y] points in ego frame")

In [ ]:
# ============================================================================
# FRENET MAP: Map representation in Frenet coordinates
# ============================================================================
# Frenet frame: (s, d) where:
#   - s: Distance along the path (longitudinal, like a road marker)
#   - d: Lateral offset from path (positive = left, negative = right)
# This makes it easy to compare points at the same position along the track!
frenet_map = FrenetMap(track)

print("✓ Frenet map created")
print(f"  Total track length: {frenet_map.frenet_frame.total_length:.2f}m")
print("  Frenet coordinates make it easy to:")
print("    - Find ground truth bounds at any distance s along the track")
print("    - Compare perceived vs ground truth at the same s position")
print("    - Calculate lateral deviations (d values)")

### Visualization: Understanding Frenet Coordinates

In [ ]:
# Visualize Frenet coordinates on the track
fig, ax = plt.subplots(figsize=(12, 8))

# Plot track
ax.plot(track.inner_bound[:, 0], track.inner_bound[:, 1], 'k-', linewidth=2, label='Track Boundaries')
ax.plot(track.outer_bound[:, 0], track.outer_bound[:, 1], 'k-', linewidth=2)
ax.plot(track.centerline[:, 0], track.centerline[:, 1], 'b--', linewidth=1.5, alpha=0.7, label='Centerline (s=0 to s=length)')

# Show some s markers along the path
for s_marker in [0, 20, 40, 60, 80, 100]:
    if s_marker < frenet_map.frenet_frame.total_length:
        point = frenet_map.frenet_frame.frenet_to_global(s_marker, 0)
        ax.plot(point[0], point[1], 'go', markersize=10, zorder=5)
        ax.text(point[0]+1, point[1]+1, f's={s_marker}m', fontsize=9, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Show d offsets at one point
s_example = 30.0
d_inner, d_outer = frenet_map.get_bounds_at_s(s_example)
point_center = frenet_map.frenet_frame.frenet_to_global(s_example, 0)
point_inner = frenet_map.frenet_frame.frenet_to_global(s_example, d_inner)
point_outer = frenet_map.frenet_frame.frenet_to_global(s_example, d_outer)

ax.plot([point_center[0], point_inner[0]], [point_center[1], point_inner[1]], 'r-', linewidth=2, label=f'Lateral offset d (at s={s_example}m)')
ax.plot([point_center[0], point_outer[0]], [point_center[1], point_outer[1]], 'r-', linewidth=2)
ax.plot(point_center[0], point_center[1], 'bo', markersize=12, label='Centerline (d=0)')
ax.plot(point_inner[0], point_inner[1], 'ro', markersize=10, label=f'Inner bound (d={d_inner:.2f}m)')
ax.plot(point_outer[0], point_outer[1], 'ro', markersize=10, label=f'Outer bound (d={d_outer:.2f}m)')

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('Understanding Frenet Coordinates: (s, d)')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.legend(loc='upper left', fontsize=9)
plt.tight_layout()
plt.show()

print("\nFrenet coordinate visualization:")
print("  - Green dots: s markers (distance along path)")
print(f"  - Red line: Shows lateral offset d at s={s_example}m")
print(f"  - d_inner={d_inner:.2f}m (left boundary), d_outer={d_outer:.2f}m (right boundary)")

## Step 2: Getting Perception Data - What Does the Sensor See?

In [ ]:
# Get perception points from the sensor
# This simulates what a real sensor would return: a point cloud of track boundaries
perception_points = sensor.get_perception_point_cloud_car_frame(car.state)

print(f"✓ Got {len(perception_points)} perception points")
print(f"  Points are in frame: {perception_points.frame}")
print("  Frame 'ego' means: car-centered coordinates (x=forward, y=left)")
print(f"  X range: [{perception_points.points[:, 0].min():.2f}, {perception_points.points[:, 0].max():.2f}]m")
print(f"  Y range: [{perception_points.points[:, 1].min():.2f}, {perception_points.points[:, 1].max():.2f}]m")

### Visualization: Ground Truth vs Perception

In [ ]:
# Visualize what the sensor sees vs ground truth
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# World frame view
ax1.set_title('World Frame: Ground Truth vs Perception', fontsize=14, fontweight='bold')
ax1.plot(track.inner_bound[:, 0], track.inner_bound[:, 1], 'k-', linewidth=2.5, label='Ground Truth (Inner)', alpha=0.8)
ax1.plot(track.outer_bound[:, 0], track.outer_bound[:, 1], 'k-', linewidth=2.5, label='Ground Truth (Outer)', alpha=0.8)

# Perception in world frame
if len(perception_points) > 0:
    perception_global = perception_points.to_global_frame(car.state)
    ax1.scatter(perception_global.points[:, 0], perception_global.points[:, 1],
                c='red', s=15, alpha=0.6, label='Perception (Sensor Data)', marker='.')

# Car position
car_corners = car.get_corners()
from matplotlib.patches import Polygon

car_poly = Polygon(car_corners, closed=True, color='blue', alpha=0.7, label='Car')
ax1.add_patch(car_poly)

ax1.set_xlabel('X (m)')
ax1.set_ylabel('Y (m)')
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Car frame view (what the car 'sees')
ax2.set_title('Car Frame: What the Sensor Sees', fontsize=14, fontweight='bold')
if len(perception_points) > 0:
    ax2.scatter(perception_points.points[:, 0], perception_points.points[:, 1],
                c='red', s=15, alpha=0.6, label='Perception Points', marker='.')

# Car at origin
car_corners_car = np.array([[-2, -0.9], [-2, 0.9], [2, 0.9], [2, -0.9]])
car_poly_car = Polygon(car_corners_car, closed=True, color='blue', alpha=0.7, label='Car (at origin)')
ax2.add_patch(car_poly_car)

ax2.set_xlabel('Forward (m)')
ax2.set_ylabel('Left (m)')
ax2.set_aspect('equal')
ax2.set_xlim(-45, 45)
ax2.set_ylim(-45, 45)
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

print("\nVisualization shows:")
print("  Left: World frame - see how perception (red dots) compares to ground truth (black lines)")
print("  Right: Car frame - what the sensor actually returns (360° point cloud)")

## Step 3: Understanding Frenet Frame Conversion

The key insight: We need to convert perception points to Frenet coordinates (s, d) where:
- **s**: Distance along the path (longitudinal)
- **d**: Lateral offset from the path (positive = left, negative = right)

This allows us to easily compare perceived boundaries with ground truth at the same position along the track.

In [ ]:
# Convert a few points to Frenet frame to see how it works
print("Example conversions (first 5 points):")
print("  Format: ego(x, y) -> global(x, y) -> Frenet(s, d)")
print()

for i, point in enumerate(perception_points.points[:5]):
    # Convert ego -> global -> Frenet
    point_global = car.state.transform_to_world_frame(point)
    s, d = frenet_map.frenet_frame.global_to_frenet(point_global)
    print(f"  Point {i}: ego=({point[0]:6.2f}, {point[1]:6.2f}) -> "
          f"global=({point_global[0]:6.2f}, {point_global[1]:6.2f}) -> "
          f"Frenet=(s={s:6.2f}, d={d:6.2f})")

### Visualization: Points in Different Frames

In [ ]:
# Visualize the same points in different coordinate frames
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Sample a subset of points for clarity
sample_indices = np.linspace(0, len(perception_points)-1, min(20, len(perception_points)), dtype=int)
sample_points = perception_points.points[sample_indices]

# Ego frame
axes[0].scatter(sample_points[:, 0], sample_points[:, 1], c='red', s=50, alpha=0.7)
axes[0].plot(0, 0, 'bo', markersize=15, label='Car (origin)')
axes[0].set_xlabel('Forward (m)')
axes[0].set_ylabel('Left (m)')
axes[0].set_title('Ego Frame\n(x=forward, y=left)')
axes[0].set_aspect('equal')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Global frame
sample_global = np.array([car.state.transform_to_world_frame(p) for p in sample_points])
axes[1].plot(track.inner_bound[:, 0], track.inner_bound[:, 1], 'k-', linewidth=1.5, alpha=0.5)
axes[1].plot(track.outer_bound[:, 0], track.outer_bound[:, 1], 'k-', linewidth=1.5, alpha=0.5)
axes[1].scatter(sample_global[:, 0], sample_global[:, 1], c='red', s=50, alpha=0.7)
car_corners = car.get_corners()
axes[1].add_patch(Polygon(car_corners, closed=True, color='blue', alpha=0.7))
axes[1].set_xlabel('X (m)')
axes[1].set_ylabel('Y (m)')
axes[1].set_title('Global Frame\n(world coordinates)')
axes[1].set_aspect('equal')
axes[1].grid(True, alpha=0.3)

# Frenet frame
sample_frenet = np.array([frenet_map.frenet_frame.global_to_frenet(p) for p in sample_global])
axes[2].scatter(sample_frenet[:, 0], sample_frenet[:, 1], c='red', s=50, alpha=0.7)
axes[2].set_xlabel('s (distance along path, m)')
axes[2].set_ylabel('d (lateral offset, m)')
axes[2].set_title('Frenet Frame\n(s=distance, d=lateral)')
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=0, color='b', linestyle='--', alpha=0.5, label='Centerline (d=0)')
axes[2].legend()

plt.tight_layout()
plt.show()

print("\nSame points shown in three coordinate frames:")
print("  Left: Ego frame - sensor's native output")
print("  Middle: Global frame - where points are in the world")
print("  Right: Frenet frame - aligned with the track path (easier to compare!)")

## Step 4: Getting Ground Truth Bounds

The FrenetMap provides easy access to ground truth boundaries at any distance s along the path.

In [ ]:
# Get car's current position in Frenet frame
car_pos_global = car.state.position()
s_car, d_car = frenet_map.frenet_frame.global_to_frenet(car_pos_global)
print(f"Car position in Frenet: s={s_car:.2f}, d={d_car:.2f}")
print(f"  s: {s_car:.2f}m along the track")
print(f"  d: {d_car:.2f}m lateral offset (positive = left of centerline)")

# Get ground truth bounds at car's position
d_inner, d_outer = frenet_map.get_bounds_at_s(s_car)
print(f"\nGround truth bounds at s={s_car:.2f}:")
print(f"  Inner bound (left): d={d_inner:.2f}m")
print(f"  Outer bound (right): d={d_outer:.2f}m")
print(f"  Track width: {d_outer - d_inner:.2f}m")

# Check a few points ahead
print("\nBounds at different s values (ahead of car):")
for s_offset in [0, 10, 20, 30]:
    s = s_car + s_offset
    d_inner, d_outer = frenet_map.get_bounds_at_s(s)
    print(f"  s={s:6.2f}m: d_inner={d_inner:6.2f}m, d_outer={d_outer:6.2f}m, width={d_outer-d_inner:.2f}m")

### Visualization: Ground Truth Bounds Along the Track

In [ ]:
# Visualize ground truth bounds in Frenet space
s_values = np.linspace(0, frenet_map.frenet_frame.total_length, 100)
d_inner_values = []
d_outer_values = []

for s in s_values:
    d_inner, d_outer = frenet_map.get_bounds_at_s(s)
    d_inner_values.append(d_inner)
    d_outer_values.append(d_outer)

fig, ax = plt.subplots(figsize=(12, 6))
ax.fill_between(s_values, d_inner_values, d_outer_values, alpha=0.3, color='green', label='Drivable Area')
ax.plot(s_values, d_inner_values, 'k-', linewidth=2, label='Inner Bound (d_inner)')
ax.plot(s_values, d_outer_values, 'k-', linewidth=2, label='Outer Bound (d_outer)')
ax.axhline(y=0, color='b', linestyle='--', alpha=0.5, label='Centerline (d=0)')
ax.axvline(x=s_car, color='r', linestyle='--', alpha=0.7, label=f'Car position (s={s_car:.1f}m)')
ax.set_xlabel('s (distance along path, m)')
ax.set_ylabel('d (lateral offset, m)')
ax.set_title('Ground Truth Bounds in Frenet Space')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

print("\nThis visualization shows:")
print("  - The drivable area (green) between inner and outer bounds")
print("  - How bounds vary along the track (curves cause d to change)")
print("  - Car's current position (red dashed line)")

## Step 5: Building the Alert Logic - Part 1: Convert and Filter Points

In [ ]:
def convert_perception_to_frenet(perception_points, car_state, frenet_frame):
    """
    Convert perception points to Frenet frame.

    This is the first step: transform sensor data into a coordinate system
    that's aligned with the track, making comparison easy.

    Returns:
        Array of [s, d] points in Frenet frame
    """
    frenet_points = []

    # Ensure points are in ego frame (sensor output)
    if perception_points.frame != 'ego':
        perception_points = perception_points.to_ego_frame(car_state)

    for point in perception_points.points:
        try:
            # Convert: ego -> global -> Frenet
            point_global = car_state.transform_to_world_frame(point)
            s, d = frenet_frame.global_to_frenet(point_global)
            frenet_points.append([s, d])
        except Exception:
            # Skip points that can't be converted (e.g., too far from track)
            continue

    return np.array(frenet_points) if frenet_points else np.array([]).reshape(0, 2)

# Test the conversion
frenet_points = convert_perception_to_frenet(perception_points, car.state, frenet_map.frenet_frame)
print(f"✓ Converted {len(frenet_points)} points to Frenet frame")
if len(frenet_points) > 0:
    print(f"  s range: [{frenet_points[:, 0].min():.2f}, {frenet_points[:, 0].max():.2f}]m")
    print(f"  d range: [{frenet_points[:, 1].min():.2f}, {frenet_points[:, 1].max():.2f}]m")

### Visualization: Perception Points in Frenet Frame

In [ ]:
# Visualize perception points in Frenet space
fig, ax = plt.subplots(figsize=(12, 6))

# Plot ground truth bounds
s_values = np.linspace(0, frenet_map.frenet_frame.total_length, 100)
d_inner_values = [frenet_map.get_bounds_at_s(s)[0] for s in s_values]
d_outer_values = [frenet_map.get_bounds_at_s(s)[1] for s in s_values]

ax.fill_between(s_values, d_inner_values, d_outer_values, alpha=0.2, color='green', label='Drivable Area')
ax.plot(s_values, d_inner_values, 'k-', linewidth=2, label='Ground Truth Bounds')
ax.plot(s_values, d_outer_values, 'k-', linewidth=2)

# Plot perception points
if len(frenet_points) > 0:
    ax.scatter(frenet_points[:, 0], frenet_points[:, 1], c='red', s=10, alpha=0.6, label='Perception Points')

ax.axhline(y=0, color='b', linestyle='--', alpha=0.5, label='Centerline')
ax.axvline(x=s_car, color='r', linestyle='--', alpha=0.7, label=f'Car (s={s_car:.1f}m)')

ax.set_xlabel('s (distance along path, m)')
ax.set_ylabel('d (lateral offset, m)')
ax.set_title('Perception Points in Frenet Space\n(Now we can easily compare with ground truth!)')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

print("\nNow we can see:")
print("  - Perception points (red) vs ground truth bounds (black)")
print("  - Points outside the green area are potential alerts!")
print("  - Points with large d values (far from centerline) may indicate errors")

## Step 6: Building the Alert Logic - Part 2: Filter by Lookahead Distance

In [ ]:
def filter_by_lookahead(frenet_points, s_car, lookahead_distance):
    """
    Filter points to only those within lookahead distance ahead of car.

    Why? We only care about immediate threats ahead, not behind us!
    This also reduces computation.

    Args:
        frenet_points: Array of [s, d] points
        s_car: Car's current s position
        lookahead_distance: Maximum distance ahead to consider (meters)

    Returns:
        Filtered array of [s, d] points
    """
    if len(frenet_points) == 0:
        return frenet_points

    s_end = s_car + lookahead_distance
    mask = (frenet_points[:, 0] >= s_car) & (frenet_points[:, 0] <= s_end)
    return frenet_points[mask]

# Test filtering
s_car, _ = frenet_map.frenet_frame.global_to_frenet(car.state.position())
lookahead = 20.0  # Check 20 meters ahead
filtered_points = filter_by_lookahead(frenet_points, s_car, lookahead)
print(f"✓ Filtered to {len(filtered_points)} points within {lookahead}m ahead")
print(f"  Original points: {len(frenet_points)}")
print(f"  Filtered points: {len(filtered_points)} ({100*len(filtered_points)/len(frenet_points) if len(frenet_points) > 0 else 0:.1f}%)")
if len(filtered_points) > 0:
    print(f"  s range: [{filtered_points[:, 0].min():.2f}, {filtered_points[:, 0].max():.2f}]m")

### Visualization: Lookahead Filtering

In [ ]:
# Visualize the filtering effect
fig, ax = plt.subplots(figsize=(12, 6))

# Plot all points (faded)
if len(frenet_points) > 0:
    ax.scatter(frenet_points[:, 0], frenet_points[:, 1], c='gray', s=5, alpha=0.2, label='All Points')

# Plot filtered points (highlighted)
if len(filtered_points) > 0:
    ax.scatter(filtered_points[:, 0], filtered_points[:, 1], c='red', s=20, alpha=0.7, label=f'Within {lookahead}m Lookahead')

# Plot ground truth bounds
s_values = np.linspace(0, frenet_map.frenet_frame.total_length, 100)
d_inner_values = [frenet_map.get_bounds_at_s(s)[0] for s in s_values]
d_outer_values = [frenet_map.get_bounds_at_s(s)[1] for s in s_values]
ax.fill_between(s_values, d_inner_values, d_outer_values, alpha=0.1, color='green')
ax.plot(s_values, d_inner_values, 'k-', linewidth=1.5, alpha=0.5)
ax.plot(s_values, d_outer_values, 'k-', linewidth=1.5, alpha=0.5)

# Show lookahead region
ax.axvline(x=s_car, color='blue', linestyle='-', linewidth=2, label=f'Car Position (s={s_car:.1f}m)')
ax.axvline(x=s_car + lookahead, color='orange', linestyle='--', linewidth=2, label=f'Lookahead Limit (s={s_car+lookahead:.1f}m)')
ax.axvspan(s_car, s_car + lookahead, alpha=0.1, color='yellow', label='Lookahead Region')

ax.set_xlabel('s (distance along path, m)')
ax.set_ylabel('d (lateral offset, m)')
ax.set_title(f'Lookahead Filtering: Only Checking {lookahead}m Ahead')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

print("\nFiltering visualization:")
print("  - Gray points: All perception points (ignored)")
print(f"  - Red points: Points within {lookahead}m ahead (checked for alerts)")
print("  - Yellow region: The lookahead zone we're monitoring")

## Step 7: Building the Alert Logic - Part 3: Calculate Deviations

In [ ]:
def calculate_deviations(frenet_points, frenet_map):
    """
    Calculate deviation of each point from ground truth bounds.

    For each point:
    - If d < d_inner: point is too far left (inside track) -> deviation = |d - d_inner|
    - If d > d_outer: point is too far right (outside track) -> deviation = |d - d_outer|
    - Otherwise: point is within bounds -> deviation = 0

    Returns:
        deviations: Array of deviation values (meters)
        alert_points: List of points with non-zero deviations
    """
    deviations = []
    alert_points = []

    for s, d_perceived in frenet_points:
        # Get ground truth bounds at this s
        d_inner, d_outer = frenet_map.get_bounds_at_s(s)

        # Calculate deviation
        if d_perceived < d_inner:
            # Too far left (inside track)
            deviation = abs(d_perceived - d_inner)
        elif d_perceived > d_outer:
            # Too far right (outside track)
            deviation = abs(d_perceived - d_outer)
        else:
            # Within bounds
            deviation = 0.0

        deviations.append(deviation)

        # Store alert point if there's a deviation
        if deviation > 0:
            alert_points.append({
                's': s,
                'd': d_perceived,
                'deviation': deviation,
                'd_inner': d_inner,
                'd_outer': d_outer,
            })

    return np.array(deviations), alert_points

# Test deviation calculation
deviations, alert_points = calculate_deviations(filtered_points, frenet_map)
print(f"✓ Calculated deviations for {len(deviations)} points")
print(f"  Max deviation: {np.max(deviations) if len(deviations) > 0 else 0:.3f}m")
print(f"  Mean deviation: {np.mean(deviations) if len(deviations) > 0 else 0:.3f}m")
print(f"  Points with deviation > 0: {np.sum(deviations > 0)} ({100*np.sum(deviations > 0)/len(deviations) if len(deviations) > 0 else 0:.1f}%)")
if alert_points:
    print(f"\n  First alert point: s={alert_points[0]['s']:.2f}, d={alert_points[0]['d']:.2f}, deviation={alert_points[0]['deviation']:.3f}m")

### Visualization: Deviation Calculation

In [ ]:
# Visualize deviations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Points colored by deviation
s_values = np.linspace(0, frenet_map.frenet_frame.total_length, 100)
d_inner_values = [frenet_map.get_bounds_at_s(s)[0] for s in s_values]
d_outer_values = [frenet_map.get_bounds_at_s(s)[1] for s in s_values]

ax1.fill_between(s_values, d_inner_values, d_outer_values, alpha=0.2, color='green', label='Drivable Area')
ax1.plot(s_values, d_inner_values, 'k-', linewidth=2)
ax1.plot(s_values, d_outer_values, 'k-', linewidth=2, label='Ground Truth Bounds')

if len(filtered_points) > 0:
    # Color points by deviation
    scatter = ax1.scatter(filtered_points[:, 0], filtered_points[:, 1],
                         c=deviations, s=30, alpha=0.7, cmap='Reds',
                         vmin=0, vmax=max(2.0, np.max(deviations)) if len(deviations) > 0 else 2.0)
    plt.colorbar(scatter, ax=ax1, label='Deviation (m)')

ax1.axhline(y=0, color='b', linestyle='--', alpha=0.5)
ax1.axvline(x=s_car, color='r', linestyle='--', alpha=0.7)
ax1.set_xlabel('s (distance along path, m)')
ax1.set_ylabel('d (lateral offset, m)')
ax1.set_title('Points Colored by Deviation\n(Redder = Larger Deviation)')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plot 2: Deviation histogram
if len(deviations) > 0:
    ax2.hist(deviations, bins=20, edgecolor='black', alpha=0.7)
    ax2.axvline(x=np.mean(deviations), color='orange', linestyle='--', linewidth=2, label=f'Mean: {np.mean(deviations):.3f}m')
    ax2.axvline(x=np.max(deviations), color='red', linestyle='--', linewidth=2, label=f'Max: {np.max(deviations):.3f}m')
    ax2.set_xlabel('Deviation (m)')
    ax2.set_ylabel('Number of Points')
    ax2.set_title('Deviation Distribution')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDeviation visualization:")
print("  Left: Points colored by how far they deviate from bounds")
print("  Right: Histogram showing distribution of deviations")

## Step 8: Building the Alert Logic - Part 4: Generate Alerts

In [ ]:
def generate_alerts(deviations, warning_threshold, critical_threshold):
    """
    Generate alert flags based on deviations and thresholds.

    Alert levels:
    - Normal: All deviations < warning_threshold
    - Warning: At least one deviation > warning_threshold but < critical_threshold
    - Critical: At least one deviation > critical_threshold

    Returns:
        Dictionary with alert information
    """
    if len(deviations) == 0:
        return {
            'has_warning': False,
            'has_critical': False,
            'max_deviation': 0.0,
            'mean_deviation': 0.0,
        }

    max_deviation = np.max(deviations)
    mean_deviation = np.mean(deviations)

    has_warning = max_deviation > warning_threshold
    has_critical = max_deviation > critical_threshold

    return {
        'has_warning': has_warning,
        'has_critical': has_critical,
        'max_deviation': max_deviation,
        'mean_deviation': mean_deviation,
    }

# Test alert generation
warning_threshold = 1.0  # 1 meter deviation triggers warning
critical_threshold = 2.0  # 2 meter deviation triggers critical alert
alert_info = generate_alerts(deviations, warning_threshold, critical_threshold)

print("Alert Results:")
print(f"  Has Warning: {alert_info['has_warning']}")
print(f"  Has Critical: {alert_info['has_critical']}")
print(f"  Max Deviation: {alert_info['max_deviation']:.3f}m")
print(f"  Mean Deviation: {alert_info['mean_deviation']:.3f}m")
print(f"\n  Thresholds: Warning > {warning_threshold}m, Critical > {critical_threshold}m")

### Visualization: Alert Thresholds

In [ ]:
# Visualize alerts with thresholds
fig, ax = plt.subplots(figsize=(12, 6))

# Plot deviations over s
if len(filtered_points) > 0:
    ax.scatter(filtered_points[:, 0], deviations, c=deviations, s=30, alpha=0.7,
              cmap='RdYlGn_r', vmin=0, vmax=max(3.0, np.max(deviations)) if len(deviations) > 0 else 3.0)

# Plot thresholds
ax.axhline(y=warning_threshold, color='orange', linestyle='--', linewidth=2, label=f'Warning Threshold ({warning_threshold}m)')
ax.axhline(y=critical_threshold, color='red', linestyle='--', linewidth=2, label=f'Critical Threshold ({critical_threshold}m)')

# Fill zones
ax.axhspan(0, warning_threshold, alpha=0.2, color='green', label='Normal Zone')
ax.axhspan(warning_threshold, critical_threshold, alpha=0.2, color='orange', label='Warning Zone')
ax.axhspan(critical_threshold, ax.get_ylim()[1], alpha=0.2, color='red', label='Critical Zone')

ax.set_xlabel('s (distance along path, m)')
ax.set_ylabel('Deviation (m)')
ax.set_title('Alert Thresholds and Deviations')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

print("\nAlert threshold visualization:")
print("  - Green zone: Normal (deviation < warning threshold)")
print("  - Orange zone: Warning (deviation between thresholds)")
print("  - Red zone: Critical (deviation > critical threshold)")

## Step 9: Putting It All Together - Complete Alert Function

In [ ]:
def check_track_bounds_alert(
    perception_points,
    car_state,
    frenet_map,
    warning_threshold=1.0,
    critical_threshold=2.0,
    lookahead_distance=20.0,
):
    """
    Complete track bounds alert check.

    This function combines all the steps:
    1. Convert perception to Frenet
    2. Filter by lookahead
    3. Calculate deviations
    4. Generate alerts

    This is the function you would use in your system!
    """
    # Step 1: Convert perception to Frenet
    frenet_points = convert_perception_to_frenet(
        perception_points, car_state, frenet_map.frenet_frame
    )

    if len(frenet_points) == 0:
        return {
            'has_warning': False,
            'has_critical': False,
            'max_deviation': 0.0,
            'mean_deviation': 0.0,
            'deviations': np.array([]),
            'alert_points': [],
        }

    # Step 2: Get car position in Frenet
    s_car, _ = frenet_map.frenet_frame.global_to_frenet(car_state.position())

    # Step 3: Filter by lookahead
    filtered_points = filter_by_lookahead(frenet_points, s_car, lookahead_distance)

    if len(filtered_points) == 0:
        return {
            'has_warning': False,
            'has_critical': False,
            'max_deviation': 0.0,
            'mean_deviation': 0.0,
            'deviations': np.array([]),
            'alert_points': [],
        }

    # Step 4: Calculate deviations
    deviations, alert_points = calculate_deviations(filtered_points, frenet_map)

    # Step 5: Generate alerts
    alert_info = generate_alerts(deviations, warning_threshold, critical_threshold)

    # Combine results
    return {
        **alert_info,
        'deviations': deviations,
        'alert_points': alert_points,
    }

# Test the complete function
result = check_track_bounds_alert(
    perception_points,
    car.state,
    frenet_map,
    warning_threshold=1.0,
    critical_threshold=2.0,
    lookahead_distance=20.0,
)

print("Complete Alert Check Result:")
for key, value in result.items():
    if key != 'deviations' and key != 'alert_points':
        print(f"  {key}: {value}")
print(f"  Number of alert points: {len(result['alert_points'])}")

## Step 10: Using Built-in Visualization (No Custom Code Needed!)

In [ ]:
# Create visualizer (handles all the plotting for you!)
viz = AlertVisualizer(track, frenet_map)
viz.create_figure()

# Just call plot_alert_result - it does everything!
# This shows:
#   - Full track in world frame
#   - Perception points
#   - Alert points highlighted
#   - Car position
#   - Alert status
viz.plot_alert_result(result, perception_points, car)

viz.show()

print("\n✓ Used AlertVisualizer - no custom plotting code needed!")
print("  The visualizer automatically:")
print("    - Plots track boundaries")
print("    - Shows perception points")
print("    - Highlights alert points")
print("    - Displays alert status")

## Step 11: Using in a Simulation Loop

In [ ]:
# Simulation loop with alerts
dt = 0.1  # Time step (seconds)
num_steps = 50
alert_history = []

print("Running simulation...")
for step in range(num_steps):
    # Update car (simple forward motion with slight steering)
    car.update(dt, acceleration=0.0, steering_rate=0.05)

    # Get perception (what sensor sees)
    perception_points = sensor.get_perception_point_cloud_car_frame(car.state)

    # Check alerts (using our function!)
    result = check_track_bounds_alert(
        perception_points,
        car.state,
        frenet_map,
        warning_threshold=1.0,
        critical_threshold=2.0,
    )

    # Log alerts
    if result['has_critical']:
        print(f"Step {step}: CRITICAL - Max deviation = {result['max_deviation']:.2f}m")
    elif result['has_warning']:
        print(f"Step {step}: WARNING - Max deviation = {result['max_deviation']:.2f}m")

    alert_history.append({
        'step': step,
        'max_deviation': result['max_deviation'],
        'has_warning': result['has_warning'],
        'has_critical': result['has_critical'],
    })

print(f"\n✓ Simulation complete! Total alerts: {sum(1 for h in alert_history if h['has_warning'] or h['has_critical'])}")

## Step 12: Visualizing Alert History (Built-in!)

In [ ]:
# Set thresholds for visualization
viz.warning_threshold = 1.0
viz.critical_threshold = 2.0

# Plot history (one function call!)
viz.plot_alert_history(alert_history)

print("\n✓ Alert history plotted automatically!")
print("  Shows deviation over time with warning/critical zones highlighted")

## Summary

You've learned how to build a Track Bounds Alert System:

1. ✅ **Understanding data structures**: Track, Car, Maps, Sensor, FrenetMap
2. ✅ **Getting perception data**: Sensor returns PerceptionPoints in ego frame
3. ✅ **Frenet frame conversion**: Convert points to (s, d) coordinates
4. ✅ **Filtering by lookahead**: Only check points ahead of the car
5. ✅ **Calculating deviations**: Compare perceived vs ground truth bounds
6. ✅ **Generating alerts**: Use thresholds to trigger warnings/critical alerts
7. ✅ **Using built-in visualization**: AlertVisualizer handles all plotting!

### Key Components Explained:

- **Track**: The racing track with boundaries
- **Sensor**: Simulates 360° LiDAR, returns PerceptionPoints
- **FrenetMap**: Map in Frenet coordinates for easy comparison
- **AlertVisualizer**: Built-in visualization (no custom code needed!)

### Next Steps

- The SDK provides `TrackBoundsAlert` class that implements all of this
- You can extend it with custom logic
- Use `AlertVisualizer` for all your visualization needs
- See `docs/ALERT_SYSTEM_TRACK_BOUNDS.md` for more details